# Unit Testing and Debugging
**Author: Zak Hussain**  
**Date: 4/16/2019**

This tutorial introduces you to building blocks for unit testing in Q#. Diagnosing errors and understanding the expected outcome of your code is as true in quantum computing as it is in classical computing. 

In this notebook, you will see how the Q# language and built-in libraries allow you to build/use operations and functions to test your code.

Before starting this tutorial, I recommend you complete the following notebooks:  
* [Introduction to the Qubit](../../Qubit/Qubit.ipynb)
* [Single Qubit Gates](../../SingleQubitGates/SingleQubitGates.ipynb) 
* [Multiqubit Gystems](../../MultiQubitSystems/MultiQubitSystems.ipynb)
* [Multiqubit Gates](../../MultiQubitGates/MultiQubitGates.ipynb)

For each topic, I will link you to additional readings found in the microsoft QDK docs. I will also state what you should take away from the assigned reading. 

Given that you've completed the four tutorials above, you should understand how gates are used to manipulate qubit registers. Now we can focus on the important building blocks of writing unit tests.

We will cover the following: 

* Operations and functions 
* Using the diagnostics namespace 
* Raising errors using fail and facts
* Dump Functions

At the end of the notebook, there are exercises in building unit tests. they will start relatively simple (given you understand the fundamentals of Q#) and gradually get more difficult. Spend at least 20 minutes on each problem before looking at the answers. 

For your use, here is a cheatsheet for Q# [this cheatsheet](https://github.com/microsoft/QuantumKatas/blob/master/quickref/qsharp-quick-reference.pdf).

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the exercises):

In [1]:
%package Microsoft.Quantum.Katas::0.10.1911.1607

Microsoft.Quantum.Standard::0.10.1911.1607, Microsoft.Quantum.Katas::0.10.1911.1607

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


### Operations and Functions  

For this section, access this [article](https://docs.microsoft.com/en-us/quantum/techniques/operations-and-functions) and read the following sections:  
* Defining New Operations
* Defining New Functions

From the sections above, you should answer the following:  
* What is an operation? 
* What is a function used for in Q#? 
* how does an operation differ from a function? 
* Why is Q# called a "tuple-in-tuple-out" langauge?  

As described in the reading, **operations** are used to carry out manipulations to quantum information; whereas, **Functions** are used solely for classical computation. 

Your Q# programs will largely consist of operations that act on qubit registers. Operations are composed of other operations and can call functions. It is important to note that functions do not act on qubits and cannot call operations. They are used for computing classical values (e.g., 1+1, 2x2). Later, we will see that functions have a characteristic that make them useful for embedding assertions into Q# programs.   

Let's look at an example operation. We will use the BitFlip operation, from the "Defining New Operations" section of the reading, to observe how we can construct a unit test operation. 

### <span style="color:blue">Example 1</span>: The BitFlip Operation

**Input:** A qubit in an arbitrary state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$.

**Goal:** Apply the $X$ gate to the qubit (i.e., transform the given state into $|1\rangle$ if the target qubit starts in the $|0\rangle$ state and vice versa).

In [4]:
operation BitFlip(target : Qubit) : Unit {
    X(target); 
}

BitFlip

As we see above, the BitFlip operation flips the state of an input qubit. This operation's output is the empty tuple (), as defined by the output type *Unit*. The body of our code calls the built-in Q# operation [X](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.x), which flips our target qubit.

Testing this BitFlip operation requires us to do the following:  
1. Define an test operation that will call the BitFlip operation.
2. Allocate a qubit to store our quantum state. 
3. Check that our BitFlip manipulates our quantum state correctly. 

To accomplish step 2, we can use the [*Assert*](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.assert) or [*AssertProb*](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.assertprob) operation found in the [Intrinsic](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic) namespace. In addition, using those two operations requires us to run [*Pauli measurements*](https://docs.microsoft.com/en-us/quantum/concepts/pauli-measurements) (e.g., PauliX, PauliZ) as demonstrated in the code blocks below. 

In [88]:
// run this code block, then excecute this operation by running the cell below.
operation TestBitFlipUsingAssert () : Unit {
    // allocate a qubit in the |0> state.
    using (qubit = Qubit()) 
    {   
        // flip the qubit from the |0> state to the |1> state
        BitFlip(qubit); 
        
        // ensure that the qubit results in the One state, given the effect of the PauliZ measurment.
        // TODO: after you successfuly simulated this operation by running the cell below, try changing the One to Zero.
        Assert([PauliZ], [qubit], One, "The qubit did not result in the One state given the PauliZ effect."); 
        
        // Release the qubit
        Reset(qubit); 
    }
}

TestBitFlipUsingAssert

In [89]:
%simulate TestBitFlipUsingAssert

()

Notice that we get an empty tuple after running the simulation cell above. This is good! it means our test passed. We are using an Assert that applies the PauliZ measurement effect on the qubit and the expected result is the |1> state. 

Now, go back to the Assert operation and try changing the One result to a Zero as marked in the TODO comment. After you execute the simulate cell, you should now see an error message! 

Let's answer the question of how our Assert operation is being used in Q#. We pass in a Pauli operator that describes a measurment of interest, the quantum register which we perform the measurement, the hypothetical outcome, and an error message. Since we are executing our code in simulation, the measurement we perform does not manipulate our register; however, if the assertion fails, the excecution raises a *fail* alongside our error message. 

We will see an operation clearly showing that assertions do not manipulating our register in Example 2. Also, we will summarize the basic effect of using the Pauli measurements available.

### <span style="color:blue">Example 2</span>: The H Gate

**Input:** A qubit in an arbitrary state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$.

**Goal:** Apply the $H$ gate to the qubit (i.e., create a superposition)

In [114]:
operation CreateSuperposition (q: Qubit) : Unit {
        // place the qubit in superposition, (|0> + |1>)/sqrt(2). 
        H(q); 
}

CreateSuperposition

In [121]:
operation TestSuperpositionUsingAssertProb () : Unit {
    using (qubit = Qubit()) {
        // place qubit in superposition. 
        CreateSuperposition(qubit); 
        
        // assert that the PauliX effect on the qubit in superposition will result in the |0> state with a 
        // probability success of 1.0 within a 0.0 tolerance.
        AssertProb([PauliX], [qubit], Zero, 1.0, "The qubit does not result in the Zero state given the PauliX effect.", 0.0);
        Reset(qubit); 
    }
}

TestSuperpositionUsingAssertProb

In [1]:
// todo, create an exercise question using the assert prob that will raise an error if One, 0.0 is used as the probability succes 

The given key was not present in the dictionary.


In [122]:
%simulate TestSuperpositionUsingAssertProb

()

In [134]:
// open the diagnositcs namespace in this cell to access the DumpMachine
open Microsoft.Quantum.Diagnostics; 

operation DumpSuperpositionUsingAssertProb () : Unit {
    using (qubit = Qubit()) {
        // place qubit in superposition. 
        CreateSuperposition(qubit);  
        
        // assert that the PauliX effect on the qubit in superposition will result in the |0> state with a 
        // probability success of 1.0 within a 0.0 tolerance.
        AssertProb([PauliX], [qubit], Zero, 1.0, "The qubit does not result in the Zero state given the PauliX effect.", 0.0);
        
        // Use DumpMachine to look at the state of our entire system 
        DumpMachine(); 
        Reset(qubit); 
    }
}

DumpSuperpositionUsingAssertProb

In [135]:
%simulate DumpSuperpositionUsingAssertProb

# wave function for qubits with ids (least to most significant): 0
∣0❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
∣1❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]


()

### TODO: summarize the Pauli Measurments 

further reading can be found [here](https://docs.microsoft.com/en-us/quantum/concepts/pauli-measurements).

Now, lets look at further assertion operations and functions we have available to us in the Diagnostics namespace. 

# Getting Started with Diagnostics    

In the previous section we used the *Assert* and *AssertProb* operations from the Intrinsic namespace. We used these assertions for checking the state of our qubit. The Diagnostics namespace furthers our ability to debug errors. It is composed of additional assertion operations and functions. With this, you will be able to compare operations and characterize the state of your qubits. 

For this section, access this [Diagnostics article](https://docs.microsoft.com/en-us/quantum/libraries/standard/diagnostics) and read the following sections:  
* Testing Qubit States
* Asserting Equality of Quantum Operations

From the reading listed above, you should answer the following:  
* Do assertions affect the state of our quantum  

In [ ]:
// start showing other assertions, don't worry about functions too much aside from mentioning it. as the Testing with Fact functions
// should cover it. 

In [ ]:
// TODO: Create a two operator comparison operation, do something with an
// operator and its adjoint. 

### <span style="color:blue">Example 3</span>: The AssertQubit Operation

**Input:** A qubit in state $|\psi\rangle = |0\rangle$

**Goal:** Apply the $Y$ gate to the qubit, i.e., transform the given state into $|\psi\rangle = i|1\rangle$.

In [2]:
// operation to test notice the return type indicates the operation specializations--adjoint and controlled.
operation ApplyY (q : Qubit) : Unit is Adj+Ctl {
    Y(q); 
}

ApplyY

In [11]:
// open namespace containing the AssertQubit operation 
open Microsoft.Quantum.Diagnostics;

operation TestApplyY () : Unit {
    // allocate a qubit in the |0> state
    using (qubit = Qubit()) {
        
        // apply the Y operation
        ApplyY(qubit); 
        
        // Assert the qubit is in the expected eigenstate of the Pauli Z operator. 
        AssertQubit(Zero, qubit); 
        
        // release the qubit 
        Reset(qubit); 
    }
}

TestApplyY

In [12]:
%simulate TestApplyY

Qubit in invalid state. Expecting: Zero
	Expected:	0
	Actual:	1


Now try triggering the AssertQubit fail message. If you need a hint, click <details><summary><u>here</u></summary>
> Change the Result input from One to Zero.  
> The effect of the Pauli Z measurment on a qubit in an $i|1\rangle$ state is a qubit in the -1 eigenstate of Z, which is the Result *One*.  </details>

With testing qubit states, there are often cases where assertions that coincide with the eigenstates of Pauli operatators do not work. The example in the *Testing Qubit States* section of the  [Diagnostics article](https://docs.microsoft.com/en-us/quantum/libraries/standard/diagnostics) reading, gives the example state $|\psi\rangle = (|0\rangle + e^{\frac{i\pi}{8}}|1\rangle)/\sqrt{2}$.   
It then discusses the process of identifying three independent assertions to examine the *Result* x,y,z values for the Pauli X, Y, and Z measurements, respectively. Assuming you have read this section, let's look at an example of testing using the **AssertQubitIsWithinTolerance** operation, which implements the three independent probability equations below. Note that $\alpha$ and $\beta$ are both [Complex](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.math.complex) types. 
* $P(x=Zero|\alpha, \beta)$
* $P(y=Zero|\alpha, \beta)$
* $P(z=Zero|\alpha, \beta)$

### <span style="color:blue">Example 4</span>: The AssertQubitWithinTolerance Operation

using the [Pi function](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.math.pi)  
using the [R1 operation](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.r1)  
using the [Complex type](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.math.complex)  
  
**Input:** A qubit in the state $|\psi\rangle = |0\rangle$

**Goal:** create the state $|\psi\rangle = (|0\rangle + e^{\frac{i\pi}{8}}|1\rangle)/\sqrt{2}$

In [26]:
// open Math namespace to access the PI() function 
open Microsoft.Quantum.Math;

// operation to test
operation CreateNonPauliEigenstate (qubit : Qubit) : Unit {
    // create a superpostion 
    H(qubit); 
    
    // apply a rotation about the |1> state by the given theta. 
    let theta = PI()/8.0; 
    
    // use the r1 operation to apply (e^i*theta) to the |1> state. 
    R1(theta, qubit); 
}

CreateNonPauliEigenstate

In [39]:
// use the Math namespace to access the 'Complex' type. 
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Diagnostics;

operation TestCreateNonPauliEigenstate () : Unit {
    
    // allocate a qubit in the |0> state
    using (qubit = Qubit()) {
        
        // execute the operation we want to test. 
        CreateNonPauliEigenstate(qubit); 
        
        // check that we correctly put the qubit in the (|0⟩+𝑒^𝑖𝜋/8|1⟩)/sqrt(2) state. 
        AssertQubitIsInStateWithinTolerance((Complex(0.7, 0.), Complex(0., 1.)), qubit, 1e-5);
        
        DumpMachine();
        // release qubit.
        Reset(qubit); 
    }
}

TestCreateNonPauliEigenstate

In [40]:
%simulate TestCreateNonPauliEigenstate

# wave function for qubits with ids (least to most significant): 0
∣0❭:	 0.707107 +  0.000000 i	 == 	**********           [ 0.500000 ]     --- [  0.00000 rad ]
∣1❭:	 0.653281 +  0.270598 i	 == 	**********           [ 0.500000 ]      /- [  0.39270 rad ]


()

### <span style="color:blue">Example 5</span>: The AssertOperationEqualInPlaceCompBases Operation

**Input:** A qubit in an arbitrary state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$.

**Goal:** Apply the $H$ gate to the qubit (i.e., create a superposition)

In [ ]:
// operation to test

In [ ]:
// operation to assert 

The full list of assertion operations are found [here](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics#operations).  
Let us now take a look at a few other debugging tricks we can use.

### Testing with Fact Functions  

Aside from the assertions discussed above, The Diagnostics namespace provides conditional checks in the form **facts**. A fact is a type of function that can be represented in the following forms:  
* [Fact](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.fact) 
* [EqualityWithinToleranceFact](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.equalitywithintolerancefact)
* [NearEqualityFactC](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.nearequalityfactc)
* [EqualityFactI](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.equalityfacti)

Where we saw assertions were useful for checking the states of out quantums systems, facts check the correctness of classical conditions. If the condition of a fact is false, then our program is halted, and a fail is raised. In the example below, we will look at raising a fail using a function. Following we will use one of the facts to accomplish the same task. 

### <span style="color:blue">Example 6</span>: Raising A Failure

**Input:** A register contain some number of qubits. 

**Goal:** raise fail if the register contains more than 5 qubits.

In [212]:
// open the Core namespace to access the Length function
open Microsoft.Quantum.Core;

function CheckRegisterSize(register : Qubit[]) : Unit {
    // define the max size of the register allowed.
    let maxRegisterSize = 5; 
    
    // use the Length function to get the size of the register. 
    if (Length(register) > maxRegisterSize) {
        fail "The qubit register is too large, the register should contain no more than 5 qubits."; 
    }
}

CheckRegisterSize

Now we can create an operation to execute the CheckRegisterSize function.

In [213]:
operation SimulateCheckRegisterSize () : Unit {
    
    // TODO: try changing this value to an integer greater than 5 to trigger the 
    // fail message.
    let numQubits = 5; 
    
    // allocate the qubit register containing numQubits. 
    using (qubitRegister = Qubit[numQubits]) {
        
        // call the function to check ensure the size of our register contains < than 5 qubits.
        CheckRegisterSize(qubitRegister); 
        
        // release the qubits. 
        ResetAll(qubitRegister); 
    }
}

SimulateCheckRegisterSize

Run the simulate cell to execute the operation above. Recall the empty-tuple () means the operation executed without failure. 

In [214]:
%simulate SimulateCheckRegisterSize

()

After you have successfully ran the cell above and seen the empty-tuple () displayed, try triggering the failure by changing the value of numQubits, in the SimulateCheckRegisterSize operation, to an integer greater than 5. 

We can also apply the built-in [**Fact**](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.fact) function from the Diagnostics namespace. Below we will look at an example of how to accomplish the same task above using this functon directly within the body of an operation.

In [231]:
// open the Core namespace to access the Length function
open Microsoft.Quantum.Core;
// open the Diagnostics namespace to access the fact functions
open Microsoft.Quantum.Diagnostics;

operation TestRegisterSizeLimit () : Unit {

    let numQubits = 5;
    let maxRegisterSize = 5; 
    
    using (qubitRegister = Qubit[numQubits]) {
    
        // note that only if the condition is false, then the failure will occur.
        Fact(Length(qubitRegister) <= maxRegisterSize, "The qubit register is too large, the register should contain no more than 5 qubits.");
        
        // Release the qubit.
        ResetAll(qubitRegister); 
    }
}

TestRegisterSizeLimit

In [230]:
%simulate TestRegisterSizeLimit

()

Try playing with both the numQubits and maxRegisterSize values. Try answering the question: "How does the condition passed into the Fact function differ from using a regular if condition?" *Hint: read the comments in the operation body*.  

The full list of fact-type functions are found [here](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics#functions)

### Dump Functions  

At this stage, you have already been introduced to how dump functions work. The *DumpMachine*, an example of a dump function, was introduced in the Qubit tutorial. The key point is Q# Diagnostics namespace helps us troubleshoot our quantum programs by offering two functions, [**DumpMachine**](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.dumpmachine) and [**DumpRegister**](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.dumpregister). These two functions dump the current state of our system into a file. 

Here we will look at an example of the DumpRegister. The structure of the information written to the dump is identical to that seen with the DumpMachine ([see the Qubit tutorial section on DumpMachine](../../Qubit/Qubit.ipynb)). The difference is, DumpRegister allows us to selectively choose which qubits are placed in the dump.  Let's take a look at an example. 

### <span style="color:blue">Example 7</span>: The DumpRegister

**Input:** A qubit register containing two qubits in the |00> state. 

**Goal:** Create the seperable state, $|\psi\rangle = \frac{1}{2}(|00\rangle + i|01\rangle + i|10\rangle + |11\rangle$.

**NOTE:** For this example, navigate to your "C:" folder and create a folder called "Quantum_DumpRegister".

In [173]:
open Microsoft.Quantum.Diagnostics; 

// TODO: Create example using DumpRegister
operation CreateSeperableState () : Unit {

    // allocate two qubits
    using (qubits = Qubit[2]) {
    
        // create the desired seperable state. 
        ApplyToEachA(H, qubits); 
        ApplyToEachA(S, qubits); 
        
       // identify the location to store the qubit#.txt files.
       // manually navigate to your C folder and create the Quantum_DumpRegister folder
       let filepath_q1q2 = "C:\Quantum_DumpRegister\q1q2.txt";
       let filepath_q1 = "C:\Quantum_DumpRegister\qubit1.txt";
       let filepath_q2 = "C:\Quantum_DumpRegister\qubit2.txt";
       
       // pass the location to save the file, and the qubit(s) you want to observe.
       // note that arrays are inclusive of the max index given. 
       // in this case, we passed in both the 0th and 1st element in the register.
       DumpRegister(filepath_q1q2, qubits[0 .. 1]); 
       
       // check the state of just the first qubit. 
       DumpRegister(filepath_q1, [qubits[0]]); 
       
       // check the state of just the second qubit. 
       DumpRegister(filepath_q2, [qubits[1]]); 
       
       // release all the qubits.
       ResetAll(qubits); 
    }
}

CreateSeperableState

In [174]:
%simulate CreateSeperableState

()

Once you've run the simulation cell above, navigate to the "C:\Quantum_DumpRegister\" folder. You will see the following: 

* that we have written the entire state of the two qubits to the q1q2.txt.
* the state of just qubit one to the q1.txt.  
* the state of qubit2 to the q2.txt.   

These files visually prove we correctly created the desired state. The tensor product of qubit one and qubit two give the result shown in q1q2.txt.


In the case that the qubits are entangled, DumpRegister outputs a message stating the qubit(s) you are interested is entangled with another.

In [139]:
// TODO: Create an entangled state and use the dumpRegister Using the Dump Register 

The given key was not present in the dictionary.


## Exercises

The following exercises are designed to test your understanding of the concepts you've learned so far. 
In each exercise your task is to implement an operation that applies a particular transformation to a qubit. 
Unlike the demos you have seen so far, you don't have to allocate the qubit or to put it into a certain initial state - the qubit is already allocated, prepared in some state and provided to you as an input to the operation.

### <span style="color:blue">Exercise 1</span>: The $Y$ gate

**Input:** A qubit in an arbitrary state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$.

**Goal:** Apply the $Y$ gate to the qubit, i.e., transform the given state into $i\alpha|1\rangle - i\beta|0\rangle$.

In [3]:
%kata T1_ApplyY_Test
open Microsoft.Quantum.Diagnostics;

operation ApplyY(q : Qubit) : Unit is Adj+Ctl {
    Y(q); 
}

Success!

### <span style="color:blue">Exercise 3<span>: The DumpRegister Given an Entangled State

**Input:** A qubit in an arbitrary state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$.

**Goal:** Apply the $Y$ gate to the qubit, i.e., transform the given state into $i\alpha|1\rangle - i\beta|0\rangle$.

## Conclusion

Congratulations! You have learned enough to try solving the first part of the [Basic Gates kata](../../BasicGates/BasicGates.ipynb). 
When you are done with that, you can continue to the next tutorials in the series to learn about the [multi-qubit systems](../MultiQubitSystems/MultiQubitSystems.ipynb) and the [multi-qubit gates](../MultiQubitGates/MultiQubitGates.ipynb).